### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

In [203]:
import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria aagora
        self.chegada = 0
        self.tempInicio = 0
    
        ######################################## ADICIONADO #####################################################    
        self.tempInicio = None
        self.tempFim = None
        self.tempchegada = tempoChegada
        

    def roda(self,quantum=None,tempoAtual = None): # se rodar sem quantum, o processor roda ate o fim
        
        #print(f"Processando {self.nome} com tempoAtual = {tempoAtual}")

        #ADICIONADO - tempo inicio primeiro processo
        if self.tempInicio is None:
            self.tempInicio = tempoAtual
            #print(f"tempoInicio de {self.nome} atualizado para {self.tempInicio}")
            
        # if self.tempFim is None:
        #     self.tempFim = tempoAtual
            
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S

            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
            
            
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        #return quantum, False # False se nao fez E/S
    
        
        if self.tam == 0:
            self.tempFim = tempoAtual + quantum
            #print(f"tempoFim de {self.nome} atualizado para {self.tempFim}")
        
        return quantum, False
    
    
    def __str__(self):
        #return f"Processo {self.nome}: tam = {self.tam}, tempChegada = {self.tempChegada}, tempInicio = {self.tempInicio}, tempFim = {self.tempFim}"
        return f"Processo {self.nome}: tam = {self.tam}, tempchegada = {self.tempchegada}, tempoInicio = {self.tempInicio}, tempoFim = {self.tempFim}"

    
    
class escalonadorFIFO(object): # Protótipo de escalonador de exemplo
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        # implemente aqui o que escalonador faz quando surge um novo processo pronto
        self.prontos.append(Processo)       
      
    def proximo(self):
        if self.prontos:
            return self.prontos.pop(0)
        else:
            raise IndexError("Fila Vazia")


Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

In [204]:
nprocs = 4
nomes = ['A','B','C','D']
chanceio = [0,0,0,0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([3,3,3,3])


total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceio[i],tamanho[i],0,0)) #cria uma lista procs de Processos
    print(i)

0
1
2
3


Na célula abaixo, temos uma simulação do funcionamento de um escalonador de processos. As duas configurações importantes aqui são o valor do quantum padrão (que pode ser dinamico em algoritmos mais complexos, e quantos timeslices um processo que faz e/s passa bloqueado.

Percebam que na terceira linha é instanciado o escalonador (neste caso, um round_robin). Isto foi feito assim para ser simples trocar o escalonador e repetir a simulação, bastando criar uma classe com os métodos pronto, proximo e construtor e alterar esta linha.

In [205]:
quantum = 2
tempoBloq = 1

Meuescalonador = escalonadorFIFO(procs.copy()) #troque escalonador pelo seu escalonador
bloqueados = []

#ctrl + K + ctrl C

# for c in procs:
#      print(c)
    
if not procs:
     print("A lista de processos está vazia.")

tempo = 0

random.seed(0)

while total>0:
    p = Meuescalonador.proximo()
    if(p is not None):
        rodou, _ = p.roda(tempoAtual=tempo) #adicione quantum como parâmetro, por enquanto nao temos E/S
        
        if(p.tam>0):
            Meuescalonador.pronto(p)
        total-=rodou
        tempo+=rodou
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
        
    
#for p in procs:
    #print(f"Processo {p.nome}: tam = {p.tam}, tempchegada = {p.tempchegada}, tempoInicio = {p.tempInicio}, tempoFim = {p.tempFim}")
    #return f"Processo {p.nome}: tam = {p.tam}, tempchegada = {p.tempchegada}, tempoInicio = {p.tempInicio}, tempoFim = {p.tempFim}"



        
        
        
######################################## ADICIONADO #####################################################   


tempResposta = []
tempExecucao = []

# print("Conteúdo de procs:", procs)

for p in procs:
    tempResposta.append(p.tempInicio - p.tempchegada)
    tempExecucao.append(p.tempFim - p.tempchegada)

print("Lista tempResposta:", tempResposta)
print("Lista tempExecucao:", tempExecucao)

tempRespostaMedio = sum(tempResposta)/len(tempResposta)
tempExecucaoMedio = sum(tempExecucao)/len(tempExecucao)




print("O tempo Medio de resposta foi: ", tempRespostaMedio)
print("O tempo Medio de execução foi: ", tempExecucaoMedio)


 

    

    

A  rodou por  3  timeslice, faltam  0
B  rodou por  3  timeslice, faltam  0
C  rodou por  3  timeslice, faltam  0
D  rodou por  3  timeslice, faltam  0
Lista tempResposta: [0, 3, 6, 9]
Lista tempExecucao: [3, 6, 9, 12]
O tempo Medio de resposta foi:  4.5
O tempo Medio de execução foi:  7.5


### Laboratório: ###


1 - Altere o simulador acima para calcular o tempo de execucao medio e tempo de resposta 

2 - Implemente o escalonador por FIFO e SJF e verifique seus tempos de execução e espera.

3 - Faça em outra célula uma implementação do STCF e Round Robin, alterando o p.roda() para receber o quantum


Na segunda feira (15) haverá um questionário no ColabWeb onde você deverá utilizar suas implementações para responder às perguntas. Na terça de manhã farei uma breve conferência para tirar dúvidas.